In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import polyscope as ps
ps.init()
import sys
sys.path.append("../src")
from trimesh import *

VPos, VColors, ITris = load_off("../data/meshseg/396.off")

e2tris = {}
for i in range(ITris.shape[0]):
    for k in range(3):
        i1, i2 = ITris[i, k], ITris[i, (k+1)%3]
        edge = (min(i1, i2), max(i1, i2))
        if not edge in e2tris:
            e2tris[edge] = set([])
        e2tris[edge].add(i)
e2tris = {key:tuple(value) for key, value in e2tris.items()}


import subprocess
import os
prefix=""
blossom_path = "../blossom5"
fout = open("edges.txt", "w")
fout.write("{} {}\n".format(ITris.shape[0], len(e2tris)))
for edge in e2tris.values():
    edge = sorted(edge)
    fout.write("{} {} 1\n".format(*edge))
fout.close()
filename = "{}_blossom_out.txt".format(prefix)
subprocess.call([blossom_path] + "-e edges.txt -w".split() + [filename])



FPos = get_face_centroids(VPos, ITris)
bridge_edges = np.loadtxt(filename)[1::].astype(int)
os.remove(filename)
orig_edges = set([tuple([min(i, j), max(i, j)]) for [i, j] in e2tris.values()])
cycle_edges = orig_edges.difference(set([tuple([min(i, j), max(i, j)]) for [i, j] in bridge_edges]))
bridge_edges = np.array([x for x in bridge_edges], dtype=int)
cycle_edges = np.array([x for x in cycle_edges], dtype=int)
cycles, vertices = get_cycles(cycle_edges, FPos)
spanning_bridge_edges = get_spanning_bridges(cycles, cycle_edges, bridge_edges)

#ps.init()
#ps.register_surface_mesh("mesh", VPos, ITris)
#ps.register_curve_network("bridgeedges", FPos, bridge_edges)
#ps.register_curve_network("spanningbridgeedges", FPos, spanning_bridge_edges)
#ps.register_curve_network("cycleedges", FPos, cycle_edges)
#ps.show()

#"""
X = get_hamiltonian_cycle(VPos, ITris, blossom_path=blossom_path)
N = X.shape[0]
edges = np.zeros((N+1, 2))
edges[:, 0] = np.arange(N+1)%N
edges[:, 1] = np.arange(1, N+2)%N
ps.init()
ps.register_curve_network("hamcycle", X, edges)
ps.show()
#"""